In [1]:
from sqlalchemy import create_engine
from operator import itemgetter
import json
import math
import time

import pandas as pd
import numpy as np
#import seaborn as sns
from sklearn import linear_model
from skopt import gp_minimize
import scipy as sp
import matplotlib.pyplot as plt
%matplotlib inline

np.set_printoptions(precision=6, suppress=True)

In [33]:
race_type = 'R'
# category = 'G'
# category = 'H'

bet_type = 'E'
# bet_type = 'Q'

r = 2

In [11]:
engine = create_engine('sqlite:///../../data/exotic.db')
conn = engine.connect()

dfo = pd.read_sql_table('exotic', conn)
print('{} exotics loaded!'.format(len(dfo)))

386174 exotics loaded!


In [12]:
df = dfo.loc[dfo['race_type'].isin([race_type])]
print('{} {} races for {} betting!'.format(len(df), race_type, bet_type))

386174 R races for E betting!


In [13]:
df.tail(10)

,id,race_id,runner_numbers,race_type,bet_type,res1,res2,res3,res4,num_runners,...,run4_num,run4_win_perc,run4_win_scaled,run4_win_rank,run4_place_perc,run4_place_scaled,run4_place_rank,prediction,success,dividend
386164,386165,47713,"2,4,7,3,8,5,1,6",R,E,3,4,7,2,8.0,...,None,NaN,NaN,NaN,NaN,NaN,NaN,0.013790,0,27.4
386165,386166,47713,"2,4,7,3,8,5,1,6",R,E,3,4,7,2,8.0,...,None,NaN,NaN,NaN,NaN,NaN,NaN,0.009348,0,27.4
386166,386167,47713,"2,4,7,3,8,5,1,6",R,E,3,4,7,2,8.0,...,None,NaN,NaN,NaN,NaN,NaN,NaN,0.008661,0,27.4
386167,386168,47713,"2,4,7,3,8,5,1,6",R,E,3,4,7,2,8.0,...,None,NaN,NaN,NaN,NaN,NaN,NaN,0.004337,0,27.4
386168,386169,47713,"2,4,7,3,8,5,1,6",R,E,3,4,7,2,8.0,...,None,NaN,NaN,NaN,NaN,NaN,NaN,0.002519,0,27.4
386169,386170,47713,"2,4,7,3,8,5,1,6",R,E,3,4,7,2,8.0,...,None,NaN,NaN,NaN,NaN,NaN,NaN,0.002019,0,27.4
386170,386171,47713,"2,4,7,3,8,5,1,6",R,E,3,4,7,2,8.0,...,None,NaN,NaN,NaN,NaN,NaN,NaN,0.001461,0,27.4
386171,386172,47713,"2,4,7,3,8,5,1,6",R,E,3,4,7,2,8.0,...,None,NaN,NaN,NaN,NaN,NaN,NaN,0.001101,0,27.4
386172,386173,47713,"2,4,7,3,8,5,1,6",R,E,3,4,7,2,8.0,...,None,NaN,NaN,NaN,NaN,NaN,NaN,0.000774,0,27.4
386173,386174,47713,"2,4,7,3,8,5,1,6",R,E,3,4,7,2,8.0,...,None,NaN,NaN,NaN,NaN,NaN,NaN,0.000298,0,27.4


In [14]:
# bet size
balance = 1000
bet_chunk = balance * 0.01

In [16]:
def bet_results(book, runners, bet_chunk, num_bets):
    """get results from bets"""
    place_cutoff = 3 if len(runners) >= 8 else 2

    win_diff = 0
    max_diff = 0
    prob2scales = []
    prob0scales = []
    # profit start with loss of bet made
    outcome = {
        'success': 0,
        'profit': -bet_chunk,
        'num_bets': num_bets,
        'num_runners': len(runners),
    }
    for i, runner in enumerate(runners):
        diff = abs(runner['win_scaled'] - runner[prob])
        max_diff = max(max_diff, diff)
        if int(runner['finishingPosition']) == 1:
            win_diff = diff

        if runner[bet]:
            prob2scales.append(runner[prob] / runner['win_scaled'])
            prob0scales.append(runner[prob] - runner['win_scaled'])

            if bet_type == 'W':
                if int(runner['finishingPosition']) == 1:
                    outcome['success'] = 1
                    # profit will be payout added against inital deduction
                    # winnings is the bet made times the odds
                    winnings = runner[bet] * runner['win_odds']
                    outcome['profit'] += winnings
                break
            elif bet_type == 'P':
                if 1 <= int(runner['finishingPosition']) <= place_cutoff:
                    # success is how correct each placement was
                    outcome['success'] += 1 / num_bets
                    # profit will be return added and offset against inital deduction
                    # winnings is the bet made times the odds
                    winnings = runner[bet] * runner['place_odds']
                    outcome['profit'] += winnings

    outcome['max_diff'] = max_diff
    outcome['win_diff'] = win_diff
    outcome['prob2scale_avg'] = sum(prob2scales) / len(prob2scales)
    outcome['prob2scale'] = min(prob2scales)
    outcome['prob0scale'] = min(prob0scales)
    outcome['bet_chunk'] = bet_chunk
#     outcome['runners'] = runners
    
    # add odds curve
#     probs = sorted([r['win_odds'] for r in runners if runner['win_odds']])
#     p = np.polyfit(np.arange(len(probs)), probs, 2)
#     outcome['pa'] = p[0]
#     outcome['pb'] = p[1]
#     outcome['pc'] = p[2]
    
    book.append(outcome)

In [63]:
def bet_combinations(combs, r, x):
    """create prediction box"""

#     print(combs)
    runner_numbers = combs[0]['runner_numbers'].split(',')
    print(runner_numbers)

    # start betting on all and cut off worse runner till positive outcome
    for box_size in range(len(runner_numbers), r, -1):

        # recreate smaller pool
        pool = runner_numbers[:box_size]
        pool = list(map(int, pool))
        print('pool is {} from {} box size'.format(len(pool), box_size))
        
        # get all combs for pool
        subset = [c for c in combs if c['run1_num'] in pool and c['run2_num'] in pool]
        print('subset of {} combs'.format(len(subset)))
        
        # get predictions
        preds = sorted(s['prediction'] for s in subset)
        print('predictions: {}'.format(preds))

        percs = np.percentile(preds, [5, 25, 50, 75, 95])
        print('percs {}'.format(percs))
        raise Exception('')

        if min_profit_flag and min_scaled_flag:
            # print('breaking: {} {} {} {}'.format(min_profit_flag, avg_profit_flag, num_bets_flag, min_probs2scale_flag))
            break

    else:
        # reset going back
        for r in runners:
            r[bet] = 0
        return runners, 0

    # put bets from pool into runners
    for p in pool:
        for r in runners:
            if r['runnerNumber'] == p['runnerNumber']:
                r[bet] = p[bet]
                r['{}_type'.format(bet)] = p['{}_type'.format(bet)]
                break

    return runners, num_bets
    

In [64]:
def f(x):
    book = []
    grouped = df.groupby('race_id')
    for _, group in grouped:
#         print(group)
        box = bet_combinations(group.to_dict(orient='records'), r, x)
        raise Exception('')
            
#         if num_bets:
#             bet_results(book, runners, bet_chunk, num_bets)
                        
    book_df = pd.DataFrame(book)
    #print(book_df)
    if not len(book_df):
        return 0
    profit = book_df.profit.sum()
#     if bet_type == 'W':
#         participation = (len(book_df) ** 2) / (len(dfo) ** 2)
#     else:
#         participation = 1
#     return profit * participation * -1
    return profit / len(dfo) * -1
    

print('optimizing...')
time_start = time.time()

# x0 => min profit
# x1 => min probs2scale
# fail => avg profit

x, *grid = sp.optimize.brute(f, np.s_[0:2:4j, 0.9:1.4:16j], full_output=True)

print('done: took {:.0f}s'.format((time.time() - time_start)))
# print('x: {}'.format(x))
# print('grid: {}'.format(grid))

optimizing...
['4', '7', '6', '8', '9', '10', '1', '11', '5', '3', '2']
pool is 11 from 11 box size
subset of 55 combs
predictions: [3.700212255353108e-05, 5.571650399360806e-05, 6.368078902596608e-05, 9.778876119526103e-05, 0.00011586586333578452, 0.0001426551752956584, 0.00022044235083740205, 0.0002547680342104286, 0.00030893480288796127, 0.00033505100873298943, 0.0003569005348253995, 0.0003615728346630931, 0.0004437951138243079, 0.0004742469755001366, 0.00047958537470549345, 0.0004923813394270837, 0.0005034994683228433, 0.000515639316290617, 0.0006090599345043302, 0.000661089492496103, 0.0006708403816446662, 0.0006880497676320374, 0.0007772149401716888, 0.0008014544146135449, 0.0008152762311510742, 0.0008696623262949288, 0.0010753850219771266, 0.001102961483411491, 0.0011216182028874755, 0.0011974854860454798, 0.0014086778974160552, 0.0014360659988597035, 0.0019512157887220383, 0.0029849610291421413, 0.003019974334165454, 0.003342096460983157, 0.0033938209526240826, 0.00400624237954

Exception: 

In [ ]:
print('category: {}'.format(category))
print('bet type: {}'.format(bet_type))
print('x: {}'.format(x))
df = process_strategy(dfo, bet_positive_dutch, x)
print('${:.2f} profit per race'.format(df.profit.sum() / len(dfo)))
print('{:.0f}% of races {} / {}'.format(len(df) / len(dfo) * 100, len(df), len(dfo)))

In [ ]:
df['ones'] = np.ones(len(df))
df['win_rate'] = df.success.cumsum() / df.ones.cumsum()
win_title = '{:.0f}% win rate'.format(df.win_rate.iloc[-1] * 100)
df.win_rate.plot(title=win_title, xlim=(100), ylim=(0, 1))

In [ ]:
# ROI
roi = df.profit.rolling(20).sum() / df.bet_chunk.rolling(20).sum()
roi.plot(xlim=(100), ylim=(-1, 1), alpha=0.3)
roi = df.profit.cumsum() / df.bet_chunk.cumsum()
roi_title = '{:.1f}% ROI'.format(roi.iloc[-1] * 100)
roi.plot(xlim=(100), ylim=(-1, 1), title=roi_title)

In [ ]:
# profit per num bets 
df.num_bets.plot(kind='hist', title='# bets')
groups = df.groupby('num_bets').mean()
groups.plot(kind='bar', y='success', title='Win rate per #bets')
groups.plot(kind='bar', y='profit', title='Profit per #bets')

In [ ]:
# profit per num runners
df.num_runners.hist()
groups = df.groupby('num_runners').mean()
groups.plot(kind='bar', y='success', title='Win rate per #runners')
groups.plot(kind='bar', y='profit', title='Profit per #runners')

In [ ]:
# max diff
df.win_diff.plot(kind='hist', bins=200, alpha=0.6, ylim=(0, 100), xlim=(0, 0.3))
df.max_diff.plot(kind='hist', bins=200, alpha=0.2, ylim=(0, 100), xlim=(0, 0.3))

In [ ]:
# df.profit.plot(kind='hist', bins=50, xlim=(-5), ylim=(0, 140))
df.profit.rolling(100).sum().plot(alpha=0.4)
df.profit.cumsum().plot(alpha=0.8)

In [ ]:
# bets/runners
df['bets_vs_runners'] = df.num_bets / df.num_runners
df['bets_vs_runners'].hist(bins=10)
groups = df.groupby(df['bets_vs_runners'].apply(lambda x: round(x, 3))).sum()
groups.plot(kind='line', y='profit', title='mean profit/bets_per_runners')

In [ ]:
# # pred vs scale
# df['prob_vs_scale'] = df.pr / df.num_runners
# df['bets_vs_runners'].hist(bins=10)
# groups = df.groupby(df['bets_vs_runners'].apply(lambda x: round(x, 3))).sum()
# groups.plot(kind='line', y='profit', title='mean profit/bets_per_runners')

In [ ]:
df.prob2scale.plot(kind='hist', bins=500, title='min prob2scale', xlim=(1, 1.5))
groups = df.groupby(df['prob2scale'].apply(lambda x: round(x, 1))).mean()
groups.plot(kind='bar', y='profit', title='profit / prob2scale')
groups.plot(kind='bar', y='win_rate', title='winrate / prob2scale')

groups_sum = df.groupby(df['prob2scale'].apply(lambda x: round(x, 1))).sum()
groups_sum.plot(kind='bar', y='profit', title='sum profit / prob2scale')

In [ ]:
df.prob0scale.plot(kind='hist', bins=50, title='min prob0scale')
groups = df.groupby(df['prob0scale'].apply(lambda x: round(x, 2))).mean()
groups.plot(kind='bar', y='profit', title='profit / prob0scale')
groups.plot(kind='bar', y='win_rate', title='winrate / prob0scale')

groups_sum = df.groupby(df['prob0scale'].apply(lambda x: round(x, 2))).sum()
groups_sum.plot(kind='bar', y='profit', title='total profit')